In [1]:
import pandas as pd
import numpy as np
import treetaggerwrapper
import pymongo as pym
import re
import string

from sklearn.externals import joblib

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import confusion_matrix

In [2]:
def mongo_to_df(collection, n_last_tweets=0, retweet=False):
    tweets = collection.find(filter={'text':{'$exists':True}}, 
                             projection={'_id':False}).sort('$natural',-1).limit(n_last_tweets)
    df = pd.DataFrame()
    listTweets, listCandidats, listSentiments = [], [], []
    
    for t in tweets: 
        if not retweet: # filtrage des retweets
            if 'rt @' in t['text']:
                continue

        if t['text']: # test si liste non vide
            listTweets.append(t['text'])
            try:
                listCandidats.append(t['candidat'])
            except:
                listCandidats.append(None)
            
            try:
                listSentiments.append(t['sentiment'])
            except:
                listSentiments.append(None)
    
    df['text'], df['candidat'], df['sentiment'] = listTweets, listCandidats, listSentiments
    return df


def load_tweets(client, spellcheck, label_auto, retweet=True):
    
    if spellcheck:
        collection = client.tweet.spellchecked
        df_tweets = mongo_to_df(collection, n_last_tweets=0, retweet=retweet)
        print('Correction orthographique activée.')
        if not label_auto:
            df_tweets = df_tweets[client.tweet.labelised.count():]
    else:
        collection = client.tweet.train
        df_tweets = mongo_to_df(collection, n_last_tweets=0, retweet=retweet)
        print('Correction orthographique desactivée.')
        if label_auto:
            # Base annotée automatiquement, sur la base des hashtags (uniquement des tweets positifs)
            collection = client.tweet.labelised
            df_tweets_auto = mongo_to_df(collection, n_last_tweets=0, retweet=retweet)
            print('Ajout des tweets labélisés automatiquement...')
            print('{} tweets ajoutés.'.format(df_tweets_auto['text'].count()))
            df_tweets = pd.concat([df_tweets, df_tweets_auto], axis=0, ignore_index=True)
        
    print('\n{} tweets au total récupérés pour entraînement, répartis comme suit :'.format(df_tweets['text'].count()))
    print(df_tweets['sentiment'].value_counts())
    print(df_tweets['candidat'].value_counts())
        
    return df_tweets


def process_texts(list_of_texts, pos_tag_list, stop_words):
    # Processing the tweets (POS tagging, lemmatization, spellchecking)
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')
    list_of_processed_texts = []
    
    for text in list_of_texts:
        # Etape de filtrage
        text = re.sub(r'\w*…', '', text) # mot tronqué par Twitter
        text = re.sub(r'(?:htt)\S*', '', text) # retrait des liens http
        text = re.sub(r'\n', ' ', text) # retrait des sauts de ligne
        text = re.sub(r'\xad', '-', text)
        text = re.sub(r'@\w*', '', text) # retrait des mentions @ (ne détecte pas @XXX@...)
        text = re.sub(r'\.{3,}', '...', text) # ....... => points de suspension
        text = re.sub(r'(?=\.\w)(\.)', '. ', text) # remplacer un point entre deux mots 'A.B' par 'A. B'
        text = re.sub(r'^rt.*: ', '', text) # retrait de la mention retweet
        #text = re.sub(r'\d', '', text) # retrait des chiffres
        
        tags = tagger.tag_text(text)
        try:
            tagged_text = ['{}|{}'.format(t.split('\t')[1], t.split('\t')[2]) for t in tags
                           if (t.split('\t')[2] not in stop_words
                               and t.split('\t')[1] in pos_tag_list)]
        except:
            tagged_text = ['ERREUR']
        list_of_processed_texts.append(tagged_text)
        
    return list_of_processed_texts

In [8]:
def build_Xy(df_tweets, pos_tags_to_keep, stop_words, vocab=None, drop_dups=False, min_df=5, n_grams=(1,1)):
    print('Tagging des tweets en cours...')
    tweet_list = process_texts(df_tweets['text'], pos_tags_to_keep, stops)
    print('TreeTagger a renvoye {} erreur(s).'.format(tweet_list.count('ERREUR')))
    
    # Building TF-IDF matrix
    print('Creation de la matrice de features...')
    vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', decode_error='strict',
                                 lowercase=False, use_idf=False, norm=None, binary=False, vocabulary=vocab,
                                 min_df=min_df, max_df=1.0, ngram_range=n_grams)
    
    mat = vectorizer.fit_transform([' '.join(tweet) for tweet in tweet_list])
    del tweet_list

    print('Taille du vocabulaire : {}'.format(len(vectorizer.vocabulary_)))
    X = pd.DataFrame(mat.toarray())
    del mat

    # ajout colonnes features supplémentaires
    X['#'] = np.array([t.count('#') / 2. for t in df_tweets['text']])
    X['http'] = np.array([(t.count('http') / 2.) if t.count('http') > 1 else 0 for t in df_tweets['text']])
    X['@'] = np.array([t.count('@') / 1. for t in df_tweets['text']])
    X['n_car'] = np.array([np.log(len(t))/4 / 1. for t in df_tweets['text']])
    X['n_words'] = np.array([np.log(len(t.split(' '))) / 2 for t in df_tweets['text']])
    X[':'] = np.array([t.count(':') / 1. for t in df_tweets['text']])
    X['!'] = np.array([t.count('!') / 1. for t in df_tweets['text']])
    X['?'] = np.array([t.count('?') / 1. for t in df_tweets['text']])
    X['"'] = np.array([(t.count('"') + t.count('»')) / 2. for t in df_tweets['text']])
    
    taille1 = X.shape[0]
    taille2 = X.shape[0]
    
    if 'sentiment' in df_tweets: # si les labels sont fournis
        X = pd.concat([X, df_tweets['sentiment']], axis=1)
    else: # sinon
        X['sentiment'] = np.zeros(taille1)

    if drop_dups: # on ne retirera les doublons que pour l'ensemble d'entrainement
        X.drop_duplicates(inplace=True)
        taille2 = X.shape[0]
        print('{} doublons retirés.'.format(taille1 - taille2))
        
    print('{} documents vectorisés.'.format(taille2))
    print(X[:5])

    return X.drop('sentiment', axis=1), X['sentiment'], vectorizer.vocabulary_

In [4]:
def find_best_params(X, y, model, params_to_test, test_from=1, seed=1):
    if model not in ['logistic', 'svc', 'nb', 'rf']:
        print('Il faut choisir un modèle parmi logistic, svc, rf et nb.')
        return
    
    # Building train & test sets
    X_train, X_test, y_train, y_test = train_test_split(X[test_from:], y[test_from:],
                                                        test_size = 0.2, random_state=seed)
    X_train = pd.concat([X_train, X[:test_from]], axis=0)
    y_train = pd.concat([y_train, y[:test_from]], axis=0)
    
    n_samples, vocabulaire = X.shape

    print("Répartition dans le dataset de train ({} tweets) : \n".format(len(y_train)),
          '\tNégatif : {:.1f}'.format(len(np.extract(y_train == -1, y_train)) / len(y_train) * 100),
          '%\n\tPositif : {:.1f}'.format(len(np.extract(y_train == 1, y_train)) / len(y_train) * 100), '%')
    print("Répartition dans le dataset de test ({} tweets) : \n".format(len(y_test)),
          '\tNégatif : {:.1f}'.format(len(np.extract(y_test == -1, y_test)) / len(y_test) * 100),
          '%\n\tPositif : {:.1f}'.format(len(np.extract(y_test == 1, y_test)) / len(y_test) * 100), '%')
    print('Tweets : ' + str(n_samples) + ' / ' + 'N-grams : ' + str(vocabulaire))

    # Choice of models
    if model == 'logistic':
        clf = LogisticRegression(max_iter=2000, class_weight='balanced', multi_class='ovr')
    if model == 'svc':
        clf = LinearSVC(class_weight='balanced')
    if model == 'nb' :
        clf = MultinomialNB()
    if model == 'rf' :
        clf = RandomForestClassifier(criterion='gini', max_depth=None, max_features='auto',
                                     bootstrap=True, n_jobs=-1, verbose=0, class_weight='balanced_subsample')

    gcv = GridSearchCV(clf, params_to_test, verbose=9, n_jobs=-1, cv=4, refit=True)
    gcv.fit(X_train, y_train)
    print('Les meilleurs paramètres pour {} sont {}.'.format(model, gcv.best_params_))
    
    # Fit & predict
    print('Prédiction sur l\'ensemble de test avec ces paramètres...')
    y_pred = gcv.predict(X_test)

    print('Score', np.sum(y_pred == y_test) / len(y_pred))
    print('Répartition des prédictions : \n',
          '\tNégatif : {:.1f}'.format(len(np.extract(y_pred == -1, y_pred)) / len(y_pred) * 100),
          '%\n\tPositif : {:.1f}'.format(len(np.extract(y_pred == 1, y_pred)) / len(y_pred) * 100), '%')

    # matrice de confusion
    cf = confusion_matrix(y_test, y_pred)
    recall = np.array([cf[i,i]/cf[i,:].sum() for i in range(3)])
    precision = np.array([cf[i,i]/cf[:,i].sum() for i in range(3)])
    print('\nMatrice de confusion (ligne: classe réelle, colonne: classe prédite):')
    print(cf)
    print('Recall (négatif, neutre, positif) : {:.3f}, {:.3f}, {:.3f}'.format(recall[0], recall[1], recall[2]))
    print('Précision (négatif, neutre, positif) : {:.3f}, {:.3f}, {:.3f}'.format(precision[0], precision[1], precision[2]))
    print('Score F1 : {:.3f}'.format(np.mean(2 * recall * precision / (recall + precision))))
    
    return

def fit_predict(X, y, clf, test_from=1, seed=1):
    
    # Building train & test sets
    X_train, X_test, y_train, y_test = train_test_split(X[test_from:], y[test_from:],
                                                        test_size = 0.15, random_state=seed)
    X_train = pd.concat([X_train, X[:test_from]], axis=0)
    y_train = pd.concat([y_train, y[:test_from]], axis=0)
    
    # mélange des lignes
    n_samples, vocabulaire = X.shape

    print("Répartition dans le dataset de train ({} tweets) : \n".format(len(y_train)),
          '\tNégatif : {:.1f}'.format(len(np.extract(y_train == -1, y_train)) / len(y_train) * 100),
          '%\n\tPositif : {:.1f}'.format(len(np.extract(y_train == 1, y_train)) / len(y_train) * 100), '%')
    print("Répartition dans le dataset de test ({} tweets) : \n".format(len(y_test)),
          '\tNégatif : {:.1f}'.format(len(np.extract(y_test == -1, y_test)) / len(y_test) * 100),
          '%\n\tPositif : {:.1f}'.format(len(np.extract(y_test == 1, y_test)) / len(y_test) * 100), '%')
    print('Tweets : ' + str(n_samples) + ' / ' + 'N-grams : ' + str(vocabulaire))

    
    # Fit & predict
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print('Score', np.sum(y_pred == y_test) / len(y_pred))
    print('Répartition des prédictions : \n',
          '\tNégatif : {:.1f}'.format(len(np.extract(y_pred == -1, y_pred)) / len(y_pred) * 100),
          '%\n\tPositif : {:.1f}'.format(len(np.extract(y_pred == 1, y_pred)) / len(y_pred) * 100), '%')

    # matrice de confusion
    cf = confusion_matrix(y_test, y_pred)
    recall = np.array([cf[i,i]/cf[i,:].sum() for i in range(3)])
    precision = np.array([cf[i,i]/cf[:,i].sum() for i in range(3)])
    print('\nMatrice de confusion (ligne: classe réelle, colonne: classe prédite):')
    print(cf)
    print('Recall (négatif, neutre, positif) : {:.3f}, {:.3f}, {:.3f}'.format(recall[0], recall[1], recall[2]))
    print('Précision (négatif, neutre, positif) : {:.3f}, {:.3f}, {:.3f}'.format(precision[0], precision[1], precision[2]))
    print('Score F1 : {:.3f}'.format(np.mean(2 * recall * precision / (recall + precision))))
    
    return

#### Test du TreeTagger

In [5]:
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')
tags = tagger.tag_text('mdr lol : test nombre 800,000 €')
print('Output du tagger :', tags)
tagged_text = ['{}|{}'.format(t.split('\t')[1], t.split('\t')[2]) for t in tags]
print('Création des features :', tagged_text)

Output du tagger : ['mdr\tNOM\tmdr', 'lol\tNOM\tlol', ':\tPUN\t:', 'test\tNOM\ttest', 'nombre\tNOM\tnombre', '800\tNUM\t@card@', ',\tPUN\t,', '000\tNUM\t@card@', '€\tNOM\t€']
Création des features : ['NOM|mdr', 'NOM|lol', 'PUN|:', 'NOM|test', 'NOM|nombre', 'NUM|@card@', 'PUN|,', 'NUM|@card@', 'NOM|€']


#### Pour info : combien de tweets dans les différentes bases

In [6]:
client = pym.MongoClient('localhost',27017)
df_labelised = mongo_to_df(client.tweet.labelised, retweet=True)
df_spell = mongo_to_df(client.tweet.spellchecked, retweet=True)
df_train = mongo_to_df(client.tweet.train, retweet=True)
print(df_spell.shape[0], df_train.shape[0], df_labelised.shape[0])

18888 10001 8893


#### Chargement des tweets depuis Mongo

In [5]:
client = pym.MongoClient('localhost',27017)
df = load_tweets(client, spellcheck=False, label_auto=True, retweet=True)

Correction orthographique desactivée.
Ajout des tweets labélisés automatiquement...
8893 tweets ajoutés.

18894 tweets au total récupérés pour entraînement, répartis comme suit :
-1.0    8572
 0.0    6846
 1.0    3476
Name: sentiment, dtype: int64
fillon       2709
macron       2543
le pen       2139
hamon        1044
melenchon     239
mélenchon     219
Name: candidat, dtype: int64


#### Choix des paramètres : POS tag à garder, dictionnaire de stop words

In [9]:
# Choix des POS tags à conserver
all_postags = ['ABR', 'ADJ', 'ADV', 'DET:ART', 'DET:POS', 'INT', 'KON', 'NAM', 'NOM', 'NUM', 'PRO',
                   'PRO:DEM', 'PRO:IND', 'PRO:PER', 'PRO:POS', 'PRO:REL', 'PRP', 'PRP:det', 'PUN', 'PUN:cit',
                   'SENT', 'SYM', 'VER:cond', 'VER:futu', 'VER:impe', 'VER:impf', 'VER:pper', 'VER:ppre',
                   'VER:pres', 'VER:simp', 'VER:subi', 'VER:subp']

pos_tags_to_keep = ['ADJ', 'ADV', 'NOM', 'NUM', 'PUN:cit', 'INT', 'DET:POS', 'PRO:POS', 'PRO:DEM',
                    'VER:cond', 'VER:futu', 'VER:impe', 'VER:impf',
                    'VER:pper', 'VER:ppre', 'VER:pres', 'VER:simp', 'VER:subi', 'VER:subp']

# Choix des stop words
stops = set(list('abcdefghijklmnopqrstuvwxyz'))
X, y, voc = build_Xy(df, pos_tags_to_keep, stops, drop_dups=True, min_df=3, n_grams=(1,1))

Tagging des tweets en cours...
TreeTagger a renvoye 0 erreur(s).
Creation de la matrice de features...
Taille du vocabulaire : 4884
1628 doublons retirés.
17266 documents vectorisés.
     0    1    2    3    4    5    6    7    8    9    ...        #  http  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...      0.5   0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...      0.0   0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...      0.0   0.0   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...      0.5   0.0   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...      0.0   0.0   

     @     n_car   n_words    :    !    ?    "  sentiment  
0  0.0  1.228164  1.522261  1.0  0.0  0.0  0.0        1.0  
1  0.0  0.972955  0.972955  0.0  0.0  0.0  0.0       -1.0  
2  0.0  1.235411  1.609438  1.0  0.0  0.0  0.0       -1.0  
3  1.0  1.235411  1.545521  1.0  2.0  0.0  0.0       -1.0  
4  0.0  1.170533  1.242453  1.0  0.0  2.0  0.0       -

#### Evaluation des modeles

In [10]:
# Avec Naive Bayes
print(20 * '-')
print('Essai avec Naive Bayes')
print(20 * '-')
clf = MultinomialNB(alpha=1., fit_prior=True)
fit_predict(X, y, clf, test_from=1)

# Avec regression logistique
print(20 * '-')
print('Essai avec Regression Logistique')
print(20 * '-')
clf = LogisticRegression(C=.5, max_iter=2000, class_weight='balanced', multi_class='ovr',
                        penalty='l2', dual=True)
fit_predict(X, y, clf, test_from=1)

# Avec linear SVC
print(20 * '-')
print('Essai avec LinearSVC')
print(20 * '-')
clf = LinearSVC(C=.02, class_weight='balanced')
fit_predict(X, y, clf, test_from=1)

--------------------
Essai avec Naive Bayes
--------------------
Répartition dans le dataset de train (14676 tweets) : 
 	Négatif : 48.6 %
	Positif : 19.4 %
Répartition dans le dataset de test (2590 tweets) : 
 	Négatif : 49.3 %
	Positif : 19.7 %
Tweets : 17266 / N-grams : 4893
Score 0.713513513514
Répartition des prédictions : 
 	Négatif : 51.4 %
	Positif : 16.3 %

Matrice de confusion (ligne: classe réelle, colonne: classe prédite):
[[977 241  58]
 [242 534  27]
 [111  63 337]]
Recall (négatif, neutre, positif) : 0.766, 0.665, 0.659
Précision (négatif, neutre, positif) : 0.735, 0.637, 0.799
Score F1 : 0.708
--------------------
Essai avec Regression Logistique
--------------------
Répartition dans le dataset de train (14676 tweets) : 
 	Négatif : 48.6 %
	Positif : 19.4 %
Répartition dans le dataset de test (2590 tweets) : 
 	Négatif : 49.3 %
	Positif : 19.7 %
Tweets : 17266 / N-grams : 4893
Score 0.743243243243
Répartition des prédictions : 
 	Négatif : 49.9 %
	Positif : 16.1 %

Matr

#### Grid search

In [ ]:
# # Avec Naive Bayes
# params = {'alpha': [1.]}
# find_best_params(X, y, 'nb', params, test_from=1)

# # Avec regression logistique
# params = {'penalty':['l2'], 'dual': [True], 'C' : [.5]}
# find_best_params(X, y, 'logistic', params, test_from=1)

# # Avec linear SVC
# params = {'C' : [.02, .1, 1.]}
# find_best_params(X, y, 'svc', params, test_from=1)

### Sauvegarde du modèle et du vocabulaire

In [11]:
# Sauvegarde du vocabulaire
df_voc = pd.DataFrame.from_dict(voc, orient='index')
df_voc.to_json('trained_dict.json')
voca = pd.read_json('trained_dict.json').to_dict()[0]

# Sauvegarde du modele
clf = LogisticRegression(C=.5, max_iter=2000, class_weight='balanced', multi_class='ovr',
                        penalty='l2', dual=True)
clf.fit(X, y)
joblib.dump(clf, 'trained_logistic_regression.pkl', protocol=2) # protocole compatible avec Python2

# Choisir le modèle qui performe le mieux et ne pas oublier de changer le nom du fichier .pkl

['trained_logistic_regression.pkl']

#### Test du chargement du modele (un score élevé indique le succès de l'opération)

In [12]:
model = joblib.load('trained_logistic_regression.pkl')
y_pred = model.predict(X)

print('Matrice de confusion')
print(confusion_matrix(y, y_pred))
print('\nScore : {:.3f}'.format(np.sum(y_pred==y) / len(y_pred)))

Matrice de confusion
[[7103 1178  128]
 [ 879 4529   96]
 [ 364  336 2653]]

Score : 0.827
